In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]='3'

from tqdm import tqdm 
from PIL import Image, ImageEnhance
from skimage.util import random_noise
import cv2
import numpy as np 
from diffusers import DPMSolverMultistepScheduler, DiffusionPipeline, DDIMScheduler
# from inverse_stable_diffusion import InversableStableDiffusionPipeline
from diffusers import DDIMInverseScheduler
from diffusers import StableDiffusionPipeline
from diffusers import StableDiffusionImg2ImgPipeline
import torch 
import tempfile
from torchvision import transforms
from collections import defaultdict
import hashlib

import copy
from transformers import CLIPModel, CLIPTokenizer
from inverse_stable_diffusion import InversableStableDiffusionPipeline
from diffusers import DPMSolverMultistepScheduler, DDIMScheduler
import open_clip
from optim_utils import *
from io_utils import *
from image_utils import *
from watermark import *
import re
from types import SimpleNamespace
from collections import defaultdict
from diffusers import AutoPipelineForText2Image
from diffusers import AutoPipelineForImage2Image

model_id = 'stabilityai/stable-diffusion-2-1-base'

In [2]:
if torch.cuda.is_available() : 
    device = "cuda"
else :
    device = "cpu"

print("device : ",device)


device :  cuda


In [3]:
reference_model = None 
reference_model_pretrain = None 
model_path = "stabilityai/stable-diffusion-2-1-base"
channel_copy = 1 
hw_copy = 8 
fpr = 0.000001
num = 1000
user_number = 1000000
output_path = "/raid/home/ashhar21137/watermarking2/Gaussian-Shading/watermarked_images"
gen_seed = 0 
num_inference_steps = 50 
guidance_scale = 7.5
image_length = 512
chacha = True
num_inversion_steps = 50

args = SimpleNamespace(
    reference_model = None ,
    reference_model_pretrain = None ,
    model_path = "stabilityai/stable-diffusion-2-1-base",
    channel_copy = 1 ,
    hw_copy = 8 ,
    fpr = 0.000001,
    num = 1000,
    user_number = 1000000,
    output_path = "/raid/home/ashhar21137/watermarking2/Gaussian-Shading/watermarked_images",
    gen_seed = 0 ,
    num_inference_steps = 50 ,
    guidance_scale = 7.5,
    image_length = 512,
    chacha = True,
    num_inversion_steps = 50,
    jpeg_ratio = 0.25,
    random_crop_ratio =None ,
    random_drop_ratio = None, 
    gaussian_blur_r = None, 
    median_blur_k = None, 
    resize_ratio = None, 
    gaussian_std = None, 
    sp_prob = None, 
    brightness_factor = None
)


In [4]:
parent = '/raid/home/ashhar21137/watermarking_final_tests/gaussian_shading/gs_watermarked_images_2'

wm_ids = os.listdir(parent)
print(wm_ids)

['66514', '18885', '551125', '90570', '344806', '318574', '160559', '424102', '96557', '394326', '560459', '93785', '241364', '536587', '371029', '483751', '69675', '457732', '239728', '469605', '348850', '523617', '222016', '378710', '186233', '405207', '8649', '2024', '160163', '388911', '4428', '122108', '413746', '36633', '277854', '223726', '147170', '84427', '147016', '311914', '505126', '309322', '287927', '393493', '145179', '283524', '399956', '540162', '444546', '387083', '445528', '98760', '42493', '287541', '57870', '412535', '525589', '170558', '108627', '70000', '215481', '214704', '454325', '485894', '279108', '377694', '168194', '405613', '246446', '147303', '392136', '570246', '64460', '524068', '85685', '334080', '262845', '426038', '384029', '19380', '172899', '280980', '62604', '492386', '504616', '356702', '341550', '257350', '213546', '153946', '219025', '365363', '153674', '547471', '467311', '497616', '161919', '570608', '79481', '352884', '179620', '77806', '27

In [5]:
paraphrase_model_id = 'stabilityai/stable-diffusion-xl-base-1.0'
pipeline_text2image = AutoPipelineForText2Image.from_pretrained(paraphrase_model_id, torch_dtype=torch.float16, variant="fp16", use_safetensors=True, add_watermarker=False).to(device)
pipeline = AutoPipelineForImage2Image.from_pipe(pipeline_text2image).to(device)


/raid/home/ashhar21137/anaconda3/envs/zodiac/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
def select_random_excluding_index(lst, i):
    # Remove the element at the i-th index
    excluded_list = lst[:i] + lst[i+1:]
    # Select a random element from the remaining list
    selected_element = random.choice(excluded_list)
    # Find the index of the selected element in the original list
    original_index = lst.index(selected_element)
    return selected_element, original_index


def list_excluding_index(lst, i):
    # Return a list excluding the element at the i-th index
    return lst[:i] + lst[i+1:]


In [7]:
strength_values = [0.10,0.20,0.30,0.40,0.50]

In [8]:
with open('captions_train2014_new_format.json','r') as file : 
    data = json.load(file)


In [9]:
neg_prompt = 'deformity, bad anatomy, cloned face, amputee, people in background, asymmetric, disfigured, extra limbs, text, missing legs, missing arms, Out of frame, low quality, Poorly drawn feet'


In [10]:
save_dir = '/raid/home/ashhar21137/watermarking_final_tests/gaussian_shading/gs_paraphrases'

In [11]:
scheduler = DPMSolverMultistepScheduler.from_pretrained(model_path, subfolder='scheduler')
pipe = InversableStableDiffusionPipeline.from_pretrained(
        model_path,
        scheduler=scheduler,
        torch_dtype=torch.float16,
        revision='fp16',
)
pipe.safety_checker = None
pipe = pipe.to(device)


vae/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [12]:
# class for watermark
if chacha:
    watermark = Gaussian_Shading_chacha(channel_copy, hw_copy, fpr, user_number)
else:
    #a simple implement,
    watermark = Gaussian_Shading(channel_copy, hw_copy, fpr, user_number)


In [14]:
for strength in strength_values : 
    print(f" ----- Strenght : {strength} -----")
    paraphrase_detection = defaultdict(lambda: defaultdict(dict))

    count = 1 
    for wm_id in tqdm(wm_ids) : 
        if count > 1 : break 
        wm_dir = os.path.join(parent, wm_id)

        avg_p_no_cap = 0 
        avg_p_with_cap = 0 

        avg_det_no_cap = 0 
        avg_det_with_cap = 0 

        for img in os.listdir(wm_dir) : 
            path = os.path.join(wm_dir, img)
            # print(path)
            caption_idx = re.split('[_.]',img)[3]
            # print(caption_idx)

            captions = data['annotations'][wm_id]

            paraphrase_caption, para_id = select_random_excluding_index(captions, int(caption_idx))
            # print(paraphrase_caption)

            wm_image = Image.open(path)

            gen_image = pipeline(paraphrase_caption, image=wm_image, negative_prompt=neg_prompt, strength=strength, guidance_scale=7.5).images[0]

            directory_paraphrased = f'{save_dir}/{strength}_para/{id}'
            print(f"Saving generated images at {directory_paraphrased}")
            if not os.path.exists(directory_paraphrased):
                os.makedirs(directory_paraphrased)

            paraphrased_name = f'gen_{id}_para_id_{para_id}.png'
            gen_image.save(os.path.join(directory_paraphrased, paraphrased_name))
            print(f"Paraphrased Image saved to {os.path.join(directory_paraphrased, paraphrased_name)}")

            # Ensure gen_image is a PIL Image
            if not isinstance(gen_image, Image.Image):
                gen_image = Image.fromarray(gen_image)

            # *** Testing with captions **
            print("Testing WITHOUT captions")
            tester_prompt = ''
            text_embeddings = pipe.get_text_embedding(tester_prompt)

            gen_image1 = transform_img(gen_image).unsqueeze(0).to(text_embeddings.dtype).to(device)
            gen_image_latents_w = pipe.get_image_latents(gen_image1, sample=False)
            gen_reversed_latents_w = pipe.forward_diffusion(
                latents=gen_image_latents_w,
                text_embeddings=text_embeddings,
                guidance_scale=1,
                num_inference_steps=num_inversion_steps,
            )

            gen_acc_metric_no_cap = watermark.eval_watermark(gen_reversed_latents_w)
            print(f" attack : Paraphrase | acc_metric without caption : {gen_acc_metric_no_cap}")

            print("Testing WITH captions")
            text_embeddings = pipe.get_text_embedding(paraphrase_caption)

            gen_image1 = transform_img(gen_image).unsqueeze(0).to(text_embeddings.dtype).to(device)
            gen_image_latents_w = pipe.get_image_latents(gen_image1, sample=False)
            gen_reversed_latents_w = pipe.forward_diffusion(
                latents=gen_image_latents_w,
                text_embeddings=text_embeddings,
                guidance_scale=1,
                num_inference_steps=num_inversion_steps,
            )

            gen_acc_metric_with_cap = watermark.eval_watermark(gen_reversed_latents_w)
            print(f" attack : Paraphrase | acc_metric without caption : {gen_acc_metric_with_cap}")

            paraphrase_detection[id]['with_caption']['avg_probability'] += gen_acc_metric_with_cap / len(captions) 
            paraphrase_detection[id]['with_caption']['detection_rate'] += (gen_acc_metric_with_cap > 0.9) / len(captions)

            paraphrase_detection[id]['without_caption']['avg_probability'] += gen_acc_metric_no_cap / len(captions)
            paraphrase_detection[id]['without_caption']['detection_rate'] += (gen_acc_metric_no_cap > 0.9) / len(captions)

            count +=1 

    with open(f"/raid/home/ashhar21137/watermarking_final_tests/gaussian_shading/results/gs_{strength}_paraphrased.json", "w") as file:
        json.dump(paraphrase_detection, file, indent=4)




            

 ----- Strenght : 0.1 -----


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Saving generated images at /raid/home/ashhar21137/watermarking_final_tests/gaussian_shading/gs_paraphrases/0.1_para/<built-in function id>
Paraphrased Image saved to /raid/home/ashhar21137/watermarking_final_tests/gaussian_shading/gs_paraphrases/0.1_para/<built-in function id>/gen_<built-in function id>_para_id_2.png
Testing WITHOUT captions


  0%|          | 0/50 [00:00<?, ?it/s]

TypeError: object of type 'NoneType' has no len()

In [ ]:
for wm_img in tqdm(wm_images) : 
    wm_dir = os.path.join(wm_image_dir, wm_img) 
    wm_dir_img = os.listdir(wm_dir)

    for img in wm_dir_img : 

        print(f"image : {img}")
        path = os.path.join(wm_dir,img)
        caption_idx = re.split('[_.]',img)[3]
        id = re.split('[_]',img)[0]


        captions = data['annotations'][id]

        selected_captions = select_excluding_index(captions,int(caption_idx))

        image_w = Image.open(path)

        count = 1
        for i in range(len(selected_captions)) :
            print(f"paraphrase count : {count}")
            paraphrase_image = pipeline(selected_captions[i], image=image_w, strength=0.15, guidance_scale=7.5).images

            directory_paraphrased = os.path.join(output_path, f"{id}/left_{caption_idx}")
            # print(f"Saving generated images at {directory_paraphrased}")
            if not os.path.exists(directory_paraphrased):
                os.makedirs(directory_paraphrased)

            paraphrase_name = f'gen_{id}_{selected_captions[i]}.png'
